In [1]:
import random
import string
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [2]:
fablib = fablib_manager()

In [3]:
cluster_id =''.join(random.choice(string.ascii_lowercase) for i in range(4))
cluster_name = f'hadoop-cluster-{cluster_id}'
slice = fablib.new_slice(name=cluster_name)
site = 'STAR'

In [4]:
cluster_name

'hadoop-cluster-ycos'

In [5]:
cluster_size = 5
servers = []
routers = []
for i in range(cluster_size):
    servers.append(f'svr{i}')
    routers.append(f'rtr{i}')

        
nodes = []
nodes.extend(servers)
nodes.extend(routers)

In [6]:
for node in nodes:
    slice.add_node(name=node, site=site, cores=4, ram=8, disk=20, image='default_ubuntu_20')

In [7]:
networks = {}

for i in range(cluster_size):
    net_details = {}
    name = f'net-{servers[i]}-{routers[i]}'
    net_details['subnet'] = f"10.{i}.0.0/24"
    net_details['nodes'] = [servers[i], routers[i]]
    net_details['iface-addr']= [f"10.{i}.0.2", f"10.{i}.0.1"]
    net_details['iface-name']=[f'{servers[i]}-{name}-p1', f'{routers[i]}-{name}-p1']
    networks[name] = net_details

nxt = 0
for i in range(cluster_size):
    for j in range(i+1, cluster_size):
        net_details = {}
        name = f'net-{routers[i]}-{routers[j]}'
        net_details['subnet'] = f"10.{100+nxt}.0.0/24"
        net_details['nodes'] = [routers[i], routers[j]]
        net_details['iface-addr']= [f"10.{100+nxt}.0.{11}", f"10.{100+nxt}.0.{21}"]
        net_details['iface-name']=[f'{routers[i]}-{name}-p1', f'{routers[j]}-{name}-p1']
        nxt += 1
        networks[name] = net_details

In [8]:
networks

{'net-svr0-rtr0': {'subnet': '10.0.0.0/24',
  'nodes': ['svr0', 'rtr0'],
  'iface-addr': ['10.0.0.2', '10.0.0.1'],
  'iface-name': ['svr0-net-svr0-rtr0-p1', 'rtr0-net-svr0-rtr0-p1']},
 'net-svr1-rtr1': {'subnet': '10.1.0.0/24',
  'nodes': ['svr1', 'rtr1'],
  'iface-addr': ['10.1.0.2', '10.1.0.1'],
  'iface-name': ['svr1-net-svr1-rtr1-p1', 'rtr1-net-svr1-rtr1-p1']},
 'net-svr2-rtr2': {'subnet': '10.2.0.0/24',
  'nodes': ['svr2', 'rtr2'],
  'iface-addr': ['10.2.0.2', '10.2.0.1'],
  'iface-name': ['svr2-net-svr2-rtr2-p1', 'rtr2-net-svr2-rtr2-p1']},
 'net-svr3-rtr3': {'subnet': '10.3.0.0/24',
  'nodes': ['svr3', 'rtr3'],
  'iface-addr': ['10.3.0.2', '10.3.0.1'],
  'iface-name': ['svr3-net-svr3-rtr3-p1', 'rtr3-net-svr3-rtr3-p1']},
 'net-svr4-rtr4': {'subnet': '10.4.0.0/24',
  'nodes': ['svr4', 'rtr4'],
  'iface-addr': ['10.4.0.2', '10.4.0.1'],
  'iface-name': ['svr4-net-svr4-rtr4-p1', 'rtr4-net-svr4-rtr4-p1']},
 'net-rtr0-rtr1': {'subnet': '10.100.0.0/24',
  'nodes': ['rtr0', 'rtr1'],
  'if

In [9]:
for k,v in networks.items():
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=k).get_interfaces()[0] for node in v['nodes'] ]
    slice.add_l2network(name=k, type='L2Bridge', interfaces=ifaces)

In [10]:
slice.submit()


Retry: 21, Time: 1606 sec


ID,294af1d1-77f5-459c-81bb-ba4b448d545c
Name,hadoop-cluster-ycos
Lease Expiration (UTC),2023-11-25 20:36:59 +0000
Lease Start (UTC),2023-11-24 20:37:01 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
1d2820ff-4387-40d0-9241-82881065b4f5,rtr0,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fec1:55d7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fec1:55d7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8ba7e477-6475-451c-b552-e0023ab9cb1d,rtr1,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fea7:7af5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fea7:7af5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
171b0394-9184-45af-b48e-f007390eca1f,rtr2,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe33:540a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe33:540a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5e345ee1-bbfb-4b22-a740-ea6548f3a74d,rtr3,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fedd:bce4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fedd:bce4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
caf502c7-2634-4a9e-9ec4-d1adfbec9f6b,rtr4,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe22:387b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe22:387b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d4d5a56f-43bd-4129-90a9-f1dbd24e3bad,svr0,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fee0:8a4e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fee0:8a4e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
06ff87c6-150b-46a0-84fd-b64c1306cb65,svr1,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe79:23c8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe79:23c8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a7319be1-6291-485a-a82d-4a97ac869506,svr2,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe00:7a48,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe00:7a48,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dfd6ce02-47e4-497d-ba7f-4722c5ec7fa6,svr3,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe9c:e90d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe9c:e90d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
95b2df2a-f9f3-4e0e-8a7a-7c2406ee4a14,svr4,4,8,100,default_ubuntu_20,qcow2,star-w5.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fee4:41e2,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fee4:41e2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/w

ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
ef0fe16a-71b8-4aaf-ab55-cdef84672502,net-rtr0-rtr1,L2,L2Bridge,STAR,None,None,Active,
5d52aee8-ce15-47af-8013-3a17666d1c26,net-rtr0-rtr2,L2,L2Bridge,STAR,None,None,Active,
0dd72f69-3e49-4c71-a265-a05c041887d9,net-rtr0-rtr3,L2,L2Bridge,STAR,None,None,Active,
5136a35e-85ff-423a-97a9-916b27a2bac1,net-rtr0-rtr4,L2,L2Bridge,STAR,None,None,Active,
0015cbaa-09a6-4026-9eb3-2cc8c1deb56e,net-rtr1-rtr2,L2,L2Bridge,STAR,None,None,Active,
ebbd934e-c98a-4db2-9a5b-87f6b763915e,net-rtr1-rtr3,L2,L2Bridge,STAR,None,None,Active,
52dd9489-897c-4c60-abf8-c540c5dba851,net-rtr1-rtr4,L2,L2Bridge,STAR,None,None,Active,
a223b231-2be3-405f-866e-20d46f630875,net-rtr2-rtr3,L2,L2Bridge,STAR,None,None,Active,
dc5f5904-06af-4e8a-84e4-60462411ac92,net-rtr2-rtr4,L2,L2Bridge,STAR,None,None,Active,
a78edf88-6f58-448f-994b-46f1df169825,net-rtr3-rtr4,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
rtr0-net-svr0-rtr0-p1,p1,rtr0,net-svr0-rtr0,100,config,,7A:38:56:42:4F:3B,enp11s0,enp11s0,None,4
rtr0-net-rtr0-rtr4-p1,p1,rtr0,net-rtr0-rtr4,100,config,,76:9F:DB:88:CB:F0,enp8s0,enp8s0,None,4
rtr0-net-rtr0-rtr2-p1,p1,rtr0,net-rtr0-rtr2,100,config,,7A:24:94:23:D9:7E,enp10s0,enp10s0,None,4
rtr0-net-rtr0-rtr3-p1,p1,rtr0,net-rtr0-rtr3,100,config,,7A:15:F7:FD:6A:7F,enp9s0,enp9s0,None,4
rtr0-net-rtr0-rtr1-p1,p1,rtr0,net-rtr0-rtr1,100,config,,76:27:E1:84:57:EA,enp7s0,enp7s0,None,4
rtr1-net-rtr0-rtr1-p1,p1,rtr1,net-rtr0-rtr1,100,config,,5A:FD:86:11:47:56,enp7s0,enp7s0,None,4
rtr1-net-rtr1-rtr2-p1,p1,rtr1,net-rtr1-rtr2,100,config,,5E:8B:10:27:3F:C7,enp9s0,enp9s0,None,4
rtr1-net-rtr1-rtr4-p1,p1,rtr1,net-rtr1-rtr4,100,config,,62:5F:86:5E:DE:6A,enp11s0,enp11s0,None,4
rtr1-net-rtr1-rtr3-p1,p1,rtr1,net-rtr1-rtr3,100,config,,62:26:88:E1:61:08,enp10s0,enp10s0,None,4
rtr1-net-svr1-rtr1-p1,p1,rtr1,net-svr1-rtr1,100,config,,5E:1D:CC:8E:7F:06,enp8s0,enp8s0,None,4



Time to print interfaces 1651 seconds


'294af1d1-77f5-459c-81bb-ba4b448d545c'

In [11]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [12]:
# Create interface configuration table

if_conf = {}
for v in networks.values():
    n1 = v['iface-name'][0]
    n2 = v['iface-name'][1]
    d1 = {}
    d2 = {}
    d1['addr'] = v['iface-addr'][0]
    d1['subnet'] = v['subnet']
    d2['addr'] = v['iface-addr'][1]
    d2['subnet'] = v['subnet']
    
    if_conf[n1] = d1
    if_conf[n2] = d2    

In [13]:
if_conf

{'svr0-net-svr0-rtr0-p1': {'addr': '10.0.0.2', 'subnet': '10.0.0.0/24'},
 'rtr0-net-svr0-rtr0-p1': {'addr': '10.0.0.1', 'subnet': '10.0.0.0/24'},
 'svr1-net-svr1-rtr1-p1': {'addr': '10.1.0.2', 'subnet': '10.1.0.0/24'},
 'rtr1-net-svr1-rtr1-p1': {'addr': '10.1.0.1', 'subnet': '10.1.0.0/24'},
 'svr2-net-svr2-rtr2-p1': {'addr': '10.2.0.2', 'subnet': '10.2.0.0/24'},
 'rtr2-net-svr2-rtr2-p1': {'addr': '10.2.0.1', 'subnet': '10.2.0.0/24'},
 'svr3-net-svr3-rtr3-p1': {'addr': '10.3.0.2', 'subnet': '10.3.0.0/24'},
 'rtr3-net-svr3-rtr3-p1': {'addr': '10.3.0.1', 'subnet': '10.3.0.0/24'},
 'svr4-net-svr4-rtr4-p1': {'addr': '10.4.0.2', 'subnet': '10.4.0.0/24'},
 'rtr4-net-svr4-rtr4-p1': {'addr': '10.4.0.1', 'subnet': '10.4.0.0/24'},
 'rtr0-net-rtr0-rtr1-p1': {'addr': '10.100.0.11', 'subnet': '10.100.0.0/24'},
 'rtr1-net-rtr0-rtr1-p1': {'addr': '10.100.0.21', 'subnet': '10.100.0.0/24'},
 'rtr0-net-rtr0-rtr2-p1': {'addr': '10.101.0.11', 'subnet': '10.101.0.0/24'},
 'rtr2-net-rtr0-rtr2-p1': {'addr': '

In [14]:
from ipaddress import ip_address, IPv4Address, IPv4Network

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))

In [15]:
for r in routers:
    slice.get_node(name=r).execute("sudo sysctl -w net.ipv4.ip_forward=1")

net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1


In [16]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [17]:
# Create route configuration table

server_ips = [f'10.{i}.0.2' for i in range(cluster_size)]
server_gws = [f'10.{i}.0.1' for i in range(cluster_size)]
dest_addrs = [f'10.{i}.0.0/24' for i in range(cluster_size)]

rt_conf = []

# create paths for servers
for i in range(cluster_size):
    for j in range(cluster_size):
        if i != j:
            server_route = {}
            server_route['source'] = servers[i]
            server_route['dest'] = dest_addrs[j]
            server_route['gw'] = server_gws[i]
            rt_conf.append(server_route)
            
# Create paths for routers
for k, v in if_conf.items():
    router_route = {}
    dest_n, _, n1, n2, _ = k.split('-')
    if n1[0:3] == 'rtr' and n2[0:3] == 'rtr':
        src = (set([n1,n2]) - {dest_n}).pop()
        router_route['source'] = src
        router_route['dest'] = dest_addrs[int(dest_n[-1])]
        router_route['gw'] = v['addr']
        rt_conf.append(router_route)

In [18]:
rt_conf

[{'source': 'svr0', 'dest': '10.1.0.0/24', 'gw': '10.0.0.1'},
 {'source': 'svr0', 'dest': '10.2.0.0/24', 'gw': '10.0.0.1'},
 {'source': 'svr0', 'dest': '10.3.0.0/24', 'gw': '10.0.0.1'},
 {'source': 'svr0', 'dest': '10.4.0.0/24', 'gw': '10.0.0.1'},
 {'source': 'svr1', 'dest': '10.0.0.0/24', 'gw': '10.1.0.1'},
 {'source': 'svr1', 'dest': '10.2.0.0/24', 'gw': '10.1.0.1'},
 {'source': 'svr1', 'dest': '10.3.0.0/24', 'gw': '10.1.0.1'},
 {'source': 'svr1', 'dest': '10.4.0.0/24', 'gw': '10.1.0.1'},
 {'source': 'svr2', 'dest': '10.0.0.0/24', 'gw': '10.2.0.1'},
 {'source': 'svr2', 'dest': '10.1.0.0/24', 'gw': '10.2.0.1'},
 {'source': 'svr2', 'dest': '10.3.0.0/24', 'gw': '10.2.0.1'},
 {'source': 'svr2', 'dest': '10.4.0.0/24', 'gw': '10.2.0.1'},
 {'source': 'svr3', 'dest': '10.0.0.0/24', 'gw': '10.3.0.1'},
 {'source': 'svr3', 'dest': '10.1.0.0/24', 'gw': '10.3.0.1'},
 {'source': 'svr3', 'dest': '10.2.0.0/24', 'gw': '10.3.0.1'},
 {'source': 'svr3', 'dest': '10.4.0.0/24', 'gw': '10.3.0.1'},
 {'sourc

In [19]:
for rt in rt_conf:
    slice.get_node(name=rt['source']).ip_route_add(subnet=IPv4Network(rt['dest']), gateway=rt['gw'])

In [20]:
iface_details = []
for interface in slice.get_interfaces():
    dtl = []
    dtl.append(interface.get_name())
    dtl.append(interface.get_node().get_name())
    dtl.append(interface.get_ip_addr_ssh())
    iface_details.append(dtl)

In [21]:
iface_details

[['rtr0-net-svr0-rtr0-p1', 'rtr0', '10.0.0.1'],
 ['rtr0-net-rtr0-rtr4-p1', 'rtr0', '10.103.0.11'],
 ['rtr0-net-rtr0-rtr2-p1', 'rtr0', '10.101.0.11'],
 ['rtr0-net-rtr0-rtr3-p1', 'rtr0', '10.102.0.11'],
 ['rtr0-net-rtr0-rtr1-p1', 'rtr0', '10.100.0.11'],
 ['rtr1-net-rtr0-rtr1-p1', 'rtr1', '10.100.0.21'],
 ['rtr1-net-rtr1-rtr2-p1', 'rtr1', '10.104.0.11'],
 ['rtr1-net-rtr1-rtr4-p1', 'rtr1', '10.106.0.11'],
 ['rtr1-net-rtr1-rtr3-p1', 'rtr1', '10.105.0.11'],
 ['rtr1-net-svr1-rtr1-p1', 'rtr1', '10.1.0.1'],
 ['rtr2-net-rtr2-rtr3-p1', 'rtr2', '10.107.0.11'],
 ['rtr2-net-rtr1-rtr2-p1', 'rtr2', '10.104.0.21'],
 ['rtr2-net-svr2-rtr2-p1', 'rtr2', '10.2.0.1'],
 ['rtr2-net-rtr0-rtr2-p1', 'rtr2', '10.101.0.21'],
 ['rtr2-net-rtr2-rtr4-p1', 'rtr2', '10.108.0.11'],
 ['rtr3-net-rtr1-rtr3-p1', 'rtr3', '10.105.0.21'],
 ['rtr3-net-svr3-rtr3-p1', 'rtr3', '10.3.0.1'],
 ['rtr3-net-rtr2-rtr3-p1', 'rtr3', '10.107.0.21'],
 ['rtr3-net-rtr3-rtr4-p1', 'rtr3', '10.109.0.11'],
 ['rtr3-net-rtr0-rtr3-p1', 'rtr3', '10.102.

In [22]:
for i in range(cluster_size):
    source_server = slice.get_node(name=servers[i])
    for j in range(cluster_size):
        if (j != i):
            print(f'Ping test from {servers[i]} to {servers[j]}')
            print("")
            target_ip = slice.get_node(name=servers[j]).get_interfaces()[0].get_ip_addr()
            stdout, stderr = source_server.execute(f'ping -c 3 {target_ip}')
            print(stdout)
            print("")
            print("")


Ping test from svr0 to svr1

PING 10.1.0.2 (10.1.0.2) 56(84) bytes of data.
64 bytes from 10.1.0.2: icmp_seq=1 ttl=62 time=8.29 ms
64 bytes from 10.1.0.2: icmp_seq=2 ttl=62 time=0.259 ms
64 bytes from 10.1.0.2: icmp_seq=3 ttl=62 time=0.245 ms

--- 10.1.0.2 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2007ms
rtt min/avg/max/mdev = 0.245/2.929/8.285/3.786 ms
PING 10.1.0.2 (10.1.0.2) 56(84) bytes of data.
64 bytes from 10.1.0.2: icmp_seq=1 ttl=62 time=8.29 ms
64 bytes from 10.1.0.2: icmp_seq=2 ttl=62 time=0.259 ms
64 bytes from 10.1.0.2: icmp_seq=3 ttl=62 time=0.245 ms

--- 10.1.0.2 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2007ms
rtt min/avg/max/mdev = 0.245/2.929/8.285/3.786 ms



Ping test from svr0 to svr2

PING 10.2.0.2 (10.2.0.2) 56(84) bytes of data.
64 bytes from 10.2.0.2: icmp_seq=1 ttl=62 time=4.93 ms
64 bytes from 10.2.0.2: icmp_seq=2 ttl=62 time=0.227 ms
64 bytes from 10.2.0.2: icmp_seq=3 ttl=62 time=0.218 ms

--